In [1]:
import pandas as pd
import numpy as np

debug = True

In [2]:
class Clue(object):
    
    def __init__(self, val, ambig=None, loc= None):
        
        self.ambig = ambig
        self.val = val
        self.loc = loc
    
    
    def update_loc(self, locstart, locend):
        
        self.loc = range(locstart, locend)

        
    def __str__(self):
        return ",".join(map(str, [self.val, self.ambig, self.loc]))
    
    def __repr__(self):
        return ",".join(map(str, [self.val, self.ambig, self.loc]))
        

    
    # maybe not useful???
    def __add__(self, other):
        if type(other) is not Clue:
            return self.val + other
        return Clue(self.val + other.val,
                    self.ambig + other.ambig)

In [34]:
## clue is an object with value and ambiguity attributes
## a block is a series
# also want this to check for negative information
# e.g. known solved squares
# possible there is one solved block that makes the column/row perfect
def check_perfection(clue_list, length):

    x = len(clue_list) - 1 + sum([x.val for x in clue_list])
    
    if x == length or x < 1 : 
        return "perfect"
    
    else : 
        return "no"


In [4]:
def assign_rowcol_ambig(entry_len, entry):
    # entry is a list of Clue objects
    # updates the ambiguity value of a row/column
    ambiguity = entry_len -(len(entry) - 1 + sum([x.val for x in entry]))
    
    if debug:
        print "entry: {}, ambiguity: {}".format(entry, ambiguity)
    
    return ambiguity

In [90]:
def read_clue(fname):
    
    with open(fname) as f:

        colsplit = [map(int, x.split(","))
                        for x in
                        f.readline().strip().split(" ")]
        
        rowsplit = [map(int, x.split(","))
                        for x in
                        f.readline().strip().split(" ")]
    
    colclue = pd.Series([[Clue(val = y) for y in x] for x in colsplit])
    rowclue = pd.Series([[Clue(val = y) for y in x] for x in rowsplit])

    t = np.zeros(shape = (len(colsplit), len(rowsplit)))
    solv_mat = pd.DataFrame(np.full_like(t,
                                         fill_value = np.nan))

    rowlen = pd.Series([len(rowsplit) for x in rowsplit])
    collen = pd.Series([len(colsplit) for x in colsplit])
    
    rowambig = pd.Series([assign_rowcol_ambig(rowlen[0],entry)
                          for entry in rowclue])
    colambig = pd.Series([assign_rowcol_ambig(collen[0], entry)
                          for entry in colclue])
    

    rowprior = pd.Series([False for i in rowsplit])
    colprior = pd.Series([False for i in colsplit])
    
    rowsolved = pd.Series([False for i in rowsplit])
    colsolved = pd.Series([False for i in colsplit])
    
    col_info = pd.concat([colclue, colambig, collen, colprior, rowsolved],
                         axis = 1)
    row_info = pd.concat([rowclue, rowambig, rowlen, rowprior, rowsolved],
                         axis = 1)
    
    col_info.columns = ["cluelist", "ambig", "len", "priority", "issolved"]
    row_info.columns = ["cluelist", "ambig", "len", "priority", "issolved"]

    
    # sol_mat is empty
    # row_info is dim (4 rows, N cols), same with col_info
    return (solv_mat, row_info, col_info)

In [101]:
bool(0)

False

In [109]:
def solve(clue, solve_row):
    # JFC dat list comprehension do
    ans = "0".join(["".join(["1" for i in range(x.val)]) for x in clue.cluelist])
    return pd.Series([bool(int(i)) for i in ans])

In [110]:
solv_mat, row_info, col_info = read_clue("5x5_examp.txt")

for i, clue in row_info.iterrows():
    
    if clue.ambig == 0: 
        
        if debug:
            print solve(clue, solv_mat.iloc[i, :])
        solv_mat.iloc[i, :] = solve(clue, solv_mat.iloc[i, :])

for i, clue in col_info.iterrows():
    
    if clue.ambig == 0: 
        
        if debug:
            print solve(clue, solv_mat.iloc[:, i])
        solv_mat.iloc[:, i] = solve(clue, solv_mat.iloc[:, i])

# now we need to update ambiguity

entry: [1,None,None], ambiguity: 4
entry: [2,None,None, 1,None,None], ambiguity: 1
entry: [2,None,None, 2,None,None], ambiguity: 0
entry: [3,None,None], ambiguity: 2
entry: [1,None,None, 1,None,None], ambiguity: 2
entry: [1,None,None, 1,None,None], ambiguity: 2
entry: [2,None,None], ambiguity: 3
entry: [1,None,None, 1,None,None], ambiguity: 2
entry: [2,None,None], ambiguity: 3
entry: [5,None,None], ambiguity: 0
0     True
1     True
2    False
3     True
4     True
dtype: bool
0    True
1    True
2    True
3    True
4    True
dtype: bool


In [111]:
solv_mat

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,True
1,NaN,NaN,NaN,NaN,True
2,True,True,False,True,True
3,NaN,NaN,NaN,NaN,True
4,NaN,NaN,NaN,NaN,True


### Next Steps

* Update ambiguity based on values in the solv_mat
* Implement is_solved to determine when we are done
* Update priority based on solv_mat additions
* Update is_solved flag based on solv_mat additions

* adding logic for how to solve
* special cases like solving edges
* using non-zero ambiguity to fill in squares